<a href="https://colab.research.google.com/github/Sompote/Dino_particle/blob/main/FPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AdeelH/pytorch-fpn


Cloning into 'pytorch-fpn'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 220 (delta 26), reused 27 (delta 10), pack-reused 175
Receiving objects: 100% (220/220), 45.70 KiB | 15.23 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [2]:
%cd /content/pytorch-fpn

/content/pytorch-fpn


In [4]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from PIL import Image
from transformers import AutoImageProcessor, ViTModel,ViTFeatureExtractor,AutoModel
from transformers import Dinov2Config, Dinov2Model
import warnings
from fpn.factory import make_fpn_resnet

#torch.set_warn_always(True)
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#config=Dinov2Config(image_size=1800)
#model_di=Dinov2Model(config)

#model_di = Dinov2Model.from_pretrained("facebook/dinov2-large")

#model_di = AutoModel.from_pretrained('facebook/dinov2-large')
#process=AutoImageProcessor.from_pretrained("facebook/dinov2-large")




#model_h = ViTModel.from_pretrained("facebook/dinov2-base")
#model_h = model_h.to(device)
# Move the model to GPU if available

# Define the custom dataset
class ImageOneDDataset(Dataset):
    def __init__(self, image_dir, label_file, transform=None):
        self.image_dir = image_dir
        self.label_file = label_file
        self.transform = transform

        # Get the list of JPEG files in the root directory
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpeg')]
        # Load the labels and one-D data
        self.labels = pd.read_excel(label_file).values
        self.labels =  self.labels



    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
         # Get the image file name
        image_file = self.image_files[idx]

        # Read the image from the file
        image = Image.open(os.path.join(self.image_dir, image_file)).convert('RGB')

        # Apply transformations to the image
        if self.transform:
            image = self.transform(image)

        # Get the label and one-D data
        label =torch.tensor(self.labels).T[idx+1]/100



        # Return the image, label, and one-D data
        return image.float(), label.float()




class FPN(nn.Module):

    def __init__(self, num_classes,img_h,img_w):

        super(FPN, self).__init__()
        self.img_h=img_h
        self.img_w=img_w
        self.num_classes=num_classes
        self.model = make_fpn_resnet(
          name='resnet18',
          fpn_type='fpn',
          pretrained=True,
          num_classes=self.num_classes,
          fpn_channels=256,
          in_channels=3,
          out_size=(self.img_h, self.img_w))

        self.fc = nn.Sequential(
          nn.Linear(in_features=(self.img_h*self.img_w*self.num_classes), out_features=1000),
          nn.Sigmoid(),
          nn.Linear(in_features=1000, out_features=50),
          nn.Sigmoid(),
          nn.Linear(in_features=50, out_features=20),
          nn.Sigmoid(),
          nn.Linear(in_features=20, out_features=6),
          nn.Sigmoid())

    def forward(self, x):

        x = self.model(x)
        x = x.view(x.size(0), -1)
        x=self.fc(x)
        return x





# Create the dataset
#resize image
img_h=800
img_w=600
dataset = ImageOneDDataset(image_dir='/content/drive/MyDrive/workspace/low_fine',
                          label_file='/content/drive/MyDrive/workspace/low_fine/select_low_fine.xlsx',
                          transform=transforms.Compose([
                              transforms.Resize((img_h, img_w)),
                              transforms.ToTensor(),

                          ]))
# Split the dataset into training, validation, and test sets
train_size = int(0.9 * len(dataset))
val_size = int(0.05 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


# Create the model
model = FPN(num_classes=3,img_h=img_h,img_w=img_w)
model = model.to(device)
#model= nn.DataParallel(model,device_ids=[0,1])


criterion = nn.MSELoss()

# Define the loss function and optimizer
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train the model
for epoch in range(60):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        #model= nn.DataParallel(model,device_ids=[0])

        images = images.to(device)

        labels = labels.to(device)
        outputs = model(images)

        # Compute the loss
        loss = loss_fn(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # Update the weights
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch: {epoch+1}, Batch: {i+1}, Loss: {loss.item()}')
# Evaluate the model on the validation set
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # Move the inputs and targets to the GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            val_loss += loss_fn(outputs, targets).item()

    # Print the validation loss
    val_loss /= len(val_loader)
    print(f'Epoch: {epoch+1}, Validation Loss: {val_loss}')


# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/workspace/model.pt')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 181MB/s]


Epoch: 1, Batch: 1, Loss: 0.16559654474258423
Epoch: 1, Validation Loss: 0.07151041552424431
Epoch: 2, Batch: 1, Loss: 0.0310337096452713
Epoch: 2, Validation Loss: 0.03979512210935354
Epoch: 3, Batch: 1, Loss: 0.02676735632121563
Epoch: 3, Validation Loss: 0.03118635667487979
Epoch: 4, Batch: 1, Loss: 0.0513351708650589
Epoch: 4, Validation Loss: 0.026059717871248722
Epoch: 5, Batch: 1, Loss: 0.012762892991304398
Epoch: 5, Validation Loss: 0.02536962740123272
Epoch: 6, Batch: 1, Loss: 0.01880600117146969
Epoch: 6, Validation Loss: 0.024811766110360622
Epoch: 7, Batch: 1, Loss: 0.01068948209285736
Epoch: 7, Validation Loss: 0.0227939672768116
Epoch: 8, Batch: 1, Loss: 0.0027727168053388596
Epoch: 8, Validation Loss: 0.02339203073643148
Epoch: 9, Batch: 1, Loss: 0.0014985942980274558
Epoch: 9, Validation Loss: 0.02305656741373241
Epoch: 10, Batch: 1, Loss: 0.0065908185206353664
Epoch: 10, Validation Loss: 0.023902328219264746
Epoch: 11, Batch: 1, Loss: 0.015236495062708855
Epoch: 11, Va

In [5]:
def mape(y_true, y_pred):
    # Avoid division by zero
    y_true = torch.clamp(y_true, min=1e-8)
    # Compute the absolute percentage error
    ape = torch.abs((y_true - y_pred) / y_true)*100
    # Return the mean over all predictions
    return torch.mean(ape)

with torch.no_grad():
    test_loss = 0.0
    test_mape = 0.0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        # Move the inputs and targets to the GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        test_loss += criterion(outputs, targets).item()
        # Compute the MAPE
        test_mape += mape(targets, outputs).item()

    # Print the test loss and MAPE
    test_loss /= len(test_loader)
    test_mape /= len(test_loader)
    print(f'Test Loss: {test_loss}')
    print(f'Test MAPE: {test_mape}')

Test Loss: 0.02373806480318308
Test MAPE: 22.36635971069336
